In [6]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import geopandas as gpd
from shapely.geometry import box
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rioxarray
import xarray
from rioxarray.merge import merge_arrays
import os
from osgeo import gdal

In [7]:
bucket_name = 'opera-pst-rs-pop1'
local_path = "C:/Users/Matthew Bonnema/Documents/dswx_data/wtr/"
local_reproj = "C:/Users/Matthew Bonnema/Documents/dswx_data/wtr_reproj/"
local_dest = "C:/Users/Matthew Bonnema/Documents/dswx_data/mosaic_final/"

In [8]:
with open('dswx_manifest.txt') as f:
    lines = f.readlines()

print(lines[0])

s3://opera-pst-rs-pop1/products/OPERA_L3_DSWx_HLS_T01GDM_20220101T214749Z_20221125T202742Z_S2B_30_v0.0/



In [11]:
CONT = ['na']#['af','si','as','au','na','sa','eu'] # af,as,au,na,sa,eu
lvl = '01'

In [12]:
for cont in CONT:
    try:
        print(f'Begin mosaic for {cont}')
        print(f'****Read shapefiles')
        basin_path  = f'./shapefiles/basins/hybas_{cont}/hybas_{cont}_lev{lvl}_v1c.shp'
        basin_gdf = gpd.read_file(basin_path)
        mrg_gdf = gpd.read_file('./shapefiles/world_mgrs/mgrs_region.shp') 

        print(f'****Identify MRG tiles')
        target_basin =  basin_gdf
        grid_codes = gpd.sjoin(mrg_gdf,target_basin)
        tile_list = []
        for row in grid_codes.iterrows():
            tile_list.append(row[1].iloc[0]+row[1].iloc[1])

        print(f'****Identify raster files')
        keys = []
        for line in lines:
            prefix = line.split(bucket_name+'/')[-1][:-2]
            filename = prefix.split('/')[-1]+'_B01_WTR.tiff'#'_B06_WTR-2.tiff'#'_B01_WTR.tiff'
            tile = filename.split('_')[4][1:]
            if tile in tile_list:
                keys.append(filename)

        print(f'****Locate reprojected rasters')
        path_list = []
        reporj_list = os.listdir(local_reproj)
        for key in keys:
            if key in reporj_list:
                path_list.append(f'{local_reproj}{key}')
            else:
                print(f'{key} not found in reprojection folder')

        print(f'****Build VRT')
        vrt = gdal.BuildVRT("merged.vrt",path_list)
        print(f'****Write mosaic to file')
        gdal.Translate(f'{local_dest}{cont}_WTR.tif',vrt,options="-co COMPRESS=DEFLATE PREDICTOR=2 ZLEVEL=9")
        #gdal.Translate(f'{local_dest}{cont}_WTR2.tif',vrt,options=['COMPRESS=DEFLATE','PREDICTOR=2','ZLEVEL=9'])

        vrt = None
    except Exception as e:
        print(f'Failed to produce {cont} mosaic, error: {e}')

Begin mosaic for na
****Read shapefiles
****Identify MRG tiles
****Identify raster files
****Locate reprojected rasters
****Build VRT
****Write mosaic to file
